# Building Zero Curves

In this session we will look at how to build zero curves using OpenRedukti.

### Overview of steps

The process involves following steps:

* Create a file containing curve definitions
* Create a file containing par rates
* Start the OpenRedukti server
* Send a bootstrap request to the server
* Construct the curve objects from the raw data returned by the server

Note that we do not necessarily have to create the files above. We could supply data by populating relevant data structures. It is just more tedious work.

Currently the bootstrap function is not exposed in the Python api hence we need to send a request to the server. In future once this function is exposed in the Python api, it will not be necessary to send the request to the server. 
Note that if par sensitivities are requested then it can take quite some time to generate the curve data, hence in that case uisng a server is advisable.

### Curve Definitions
Okay so we start by creating a curve definitions file. This is just a CSV file. The example we will use can be found at [github.com/redukti/PyRedukti/blob/master/testdata/20121211/curve_definitions.csv](https://github.com/redukti/PyRedukti/blob/master/testdata/20121211/curve_definitions.csv).

Note that each curve is given a numeric id that must be unique. I will explain the curve definition parameters in a subsequent session.

### PAR Rates or market quotes
Next we need to create a par rates file. This is also a CSV file. The example we will use can be found at 
[https://github.com/redukti/PyRedukti/blob/master/testdata/20121211/par_rates.csv](https://github.com/redukti/PyRedukti/blob/master/testdata/20121211/par_rates.csv). 

Please note the use of the curve ids as the forward and discounting curve ids in the par rates data. This highlights an important aspect of OpenRedukti's curve builder: **it bootstraps multiple curves simultaneously**. Hence you need to supply all the inter-related curves together. Usually it means submitting all curves for a single currency in one request.

There are a number of details to note when creating the par rates data: I will cover these later in this session.

### Start OpenRedukti Server

The easiest way to do this is to use Docker. Docker images will be available soon for you to download and run the server with minimal effort.

Once you have the docker image, the command to start the server is:

```
docker run --rm -d -p 9001:9001/tcp redukti-grpc-server:latest
```

Above starts the service and enables port 9001.

### Checking that server is up

In [2]:
from redukti.library import ServerCommand

In [3]:
server_command = ServerCommand('localhost:9001')

In [6]:
server_command.hello('hello redukti')

'hello redukti'

The server has echoed back our input so that means it is running.

### Loading Market Data 

We can now load the curve definitions and par rates into memory structures using a library function provided by redukti.

In [7]:
from redukti.library import MarketData

In [8]:
import redukti

In [9]:
market_data = MarketData(redukti.dmy(11,12,2012)) # business date is 11th Dec 2012

In [11]:
market_data.read_curvedefinitions('../testdata/20121211/curve_definitions.csv')

In [12]:
market_data.read_parcurves('../testdata/20121211/par_rates.csv')

So now the data has been loaded and we have created the memory structures needed to submit the bootstrap request. Note that as I mentioned before, we could have done this manually rather than loading from data files.